In [1]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [2]:
import os
os.chdir('/content/drive/MyDrive/yolo/sattelite_yolo/yolov5')
# path="new_annotation"
# all_data=os.listdir(path)

In [3]:
!ls

benchmarks.py	 data	     LICENSE	  README.zh-CN.md   setup.cfg	    val.py
CITATION.cff	 detect.py   models	  requirements.txt  train.py	    yolov5l.pt
classify	 export.py   __pycache__  runs		    tutorial.ipynb  yolov5s.pt
CONTRIBUTING.md  hubconf.py  README.md	  segment	    utils


## **Inference on image**

In [ ]:
import torch
model = torch.hub.load('.', 'custom', path='/content/drive/MyDrive/yolo/sattelite_yolo/yolov5/runs/train/exp3/weights/best.pt', source='local')
# Image
img = '/content/drive/MyDrive/yolo/sattelite_yolo/data/test/t1.jpg'
# Inference
results = model(img)
# Results
results.show()  # .save()

Output hidden; open in https://colab.research.google.com to view.

## **Infernce for vedios**

predicted frames from vedios save in prediction folder of project directory

In [ ]:
import cv2
import numpy as np
# Load the video
video_path = '/content/drive/MyDrive/yolo/sattelite_yolo/Maui.MP4'  # Replace with the actual video path
cap = cv2.VideoCapture(video_path)

# Check if the video file is opened successfully
if not cap.isOpened():
    print("Error: Could not open video.")
    exit()

# Get video frame width, height, and frames per second (fps)
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

# Create a directory to save frames as images
frame_output_dir = '/content/drive/MyDrive/sattelite_yolo/prediction'
os.makedirs(frame_output_dir, exist_ok=True)
frame_count = 0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Resize the input frame
    img = cv2.resize(frame, (640, 640))
    results = model(img)  # Perform inference with YOLOv5

    # Post-process the YOLOv5 results
    results_np = results.pandas().xyxy[0].to_numpy()
    for i in range(len(results_np)):
        x0, y0, x1, y1, conf, cls = results_np[i][:6]
        box = np.array([x0, y0, x1, y1]).astype(np.int32)
        cls_id = int(cls)
        name = model.names[int(cls_id)]
        score = round(float(conf), 2)
        color = (215, 0, 40)
        cv2.rectangle(img, (box[0], box[1]), (box[2], box[3]), color, 2)
        label = f"{name}: {score}"
        label_color = (0, 0, 255)
        cv2.putText(img, label, (box[0], box[1] - 6), cv2.FONT_HERSHEY_SIMPLEX, 0.6, label_color, thickness=2)

    # Save some frames as images
    if frame_count % 50 == 0:  # Save every 50 frames (adjust as needed)
        frame_filename = os.path.join(frame_output_dir, f"frame_{frame_count}.jpg")
        cv2.imwrite(frame_filename, img)
        print(frame_count)
    frame_count += 1

# Release the video capture
cap.release()

0
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
1000
1050
1100
1150


In [ ]:
import os
from IPython.display import display, Image
# Specify the path to the folder containing saved images
folder_path = '/content/drive/MyDrive/sattelite_yolo/prediction'

# Get a list of all image files in the folder
image_files = [f for f in os.listdir(folder_path) if f.endswith('.jpg')]

# Display each image in the folder
for image_file in image_files:
    image_path = os.path.join(folder_path, image_file)
    display(Image(filename=image_path))


Output hidden; open in https://colab.research.google.com to view.